In [2]:
import os
import sys
# import re
import pickle
from collections import defaultdict, OrderedDict, Counter
import warnings
warnings.filterwarnings('ignore')

# import json
from tqdm import tqdm

In [3]:
import numpy as np
import pandas as pd
# from pandas.plotting import scatter_matrix
# from pandas_profiling import ProfileReport
# from missingno import dendrogram
# import missingno as msno

# pd.options.display.float_format = '{:0.5f}'.format`

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

# settings for seaborn plot sizes
sns.set(rc={'figure.figsize':(20, 10)})
plt.rc('font',family='Malgun Gothic')

----

# 데이터 베이스 활용

In [5]:
# 해당 데이터들의 정보와 DataFrame를 불러올 수 있는 딕셔너리
# 파일 크기 큼 주의
with open('Database.p', 'rb') as file:    
    DataBase = pickle.load(file)

In [6]:
# 해당 데이터들의 정보들만 빠르게 파악할 수 있는 딕셔너리
with open('Database_info.p', 'rb') as file:
    DataBase_info = pickle.load(file)

In [7]:
def db_NameList():
    """데이터베이스 안에 있는 data 이름들 리스트로 반환

    Returns:
        list: data Names in DataBase
    """
    return list(DataBase_info.keys())

In [8]:
def db_nameToInfo(name):
    """데이터베이스 중에 name 에 해당하는 정보들을 딕셔너리 형태로 반환

    Args:
        name (str): 데이터 이름 (한글)

    Returns:
        dict: name data's information
    """
    return DataBase_info[name]

In [9]:
def db_nameToDataframe(name):
    """데이터베이스 중에 name 에 해당하는 데이터 프레임 반환

    Args:
        name (str): 데이터 이름 (한글)

    Returns:
        DataFrame: name data's DataFrame
    """
    return DataBase[name]['data'].copy()

In [10]:
def db_nameToColumns(name):
    """데이터베이스 중에 name 에 해당하는 데이터 프레임의 칼럼 리스트 반환

    Args:
        name (str): 데이터 이름 (한글)

    Returns:
        list: name data's columns
    """
    return db_nameToInfo(name)['columns']

In [11]:
db_NameList()

['날씨별 배달 품목차이',
 '지역-업종 배달평균거리',
 '배달 상점 데이터',
 '업종 목적지별 배달 주문건수',
 '업종-거리구간 배달 건수',
 '업종 목적지별 평균주문금액',
 '주문지역 주거 특성',
 '주문지역 상권유형 특성',
 '주문지역 인구 특성',
 '주문지역 기상 정보',
 '지역-업종 결제정보',
 '지역-요일 배달 품목정보',
 '업종-거리구간 상권 정보',
 '시간-지역별 배달 주문건수',
 '시간-지역별 배달 평균주문금액',
 '시간-지역별 배달 소요시간평균',
 '업종-지역별 평균주문금액',
 '업종-지역별 배달 주문건수',
 '업종-지역별 평균배달소요시간']

In [12]:
db_nameToInfo('주문지역 상권유형 특성')

{'name': '주문지역 상권유형 특성',
 'code': 'KGUTRDARINFO',
 'len': 159651,
 'columns': ['기준연도',
  '시도명',
  '시도코드',
  '시군구명',
  '시군구코드',
  '상권업종대분류명',
  '상권업종중분류명',
  '상권업종소분류명',
  '상권업종소분류코드',
  '상권업종개수'],
 'shape': (159651, 10)}

### 0. 데이터 활용방안 모색

# 2. 주문지역 상권유형 특성 분석

In [13]:
data2=db_nameToDataframe('주문지역 상권유형 특성')

In [14]:
data2

,기준연도,시도명,시도코드,시군구명,시군구코드,상권업종대분류명,상권업종중분류명,상권업종소분류명,상권업종소분류코드,상권업종개수
0,2019,강원도,42,강릉시,42150,관광/여가/오락,PC/오락/당구/볼링등,기원,N01A09,8
1,2019,강원도,42,강릉시,42150,관광/여가/오락,PC/오락/당구/볼링등,당구장,N01A03,70
2,2019,강원도,42,강릉시,42150,관광/여가/오락,PC/오락/당구/볼링등,볼링장,N01A06,4
3,2019,강원도,42,강릉시,42150,관광/여가/오락,PC/오락/당구/볼링등,비디오감상실,N01A02,8
4,2019,강원도,42,강릉시,42150,관광/여가/오락,PC/오락/당구/볼링등,인터넷PC방,N01A01,126
...,...,...,...,...,...,...,...,...,...,...
66329,2020,충청북도,43,충주시,43130,학문/교육,학원-창업취업취미,학원-목공예,R02A14,1
66330,2020,충청북도,43,충주시,43130,학문/교육,학원-컴퓨터,컴퓨터학원,R06A01,13
66331,2020,충청북도,43,충주시,43130,학문/교육,학원기타,운전연수전문,R09A03,1
66332,2020,충청북도,43,충주시,43130,학문/교육,학원기타,학원(종합),R09A01,136


In [15]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159651 entries, 0 to 66333
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   기준연도       159651 non-null  int64 
 1   시도명        159651 non-null  object
 2   시도코드       159651 non-null  int64 
 3   시군구명       159651 non-null  object
 4   시군구코드      159651 non-null  int64 
 5   상권업종대분류명   159651 non-null  object
 6   상권업종중분류명   159651 non-null  object
 7   상권업종소분류명   159651 non-null  object
 8   상권업종소분류코드  159651 non-null  object
 9   상권업종개수     159651 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 13.4+ MB


In [17]:
data2['시도명'].unique()

array(['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시',
       '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전라북도', '제주특별자치도',
       '충청남도', '충청북도'], dtype=object)

In [20]:
data2=data2[data2['시도명']=='서울특별시'] #서울특별시만 따로!!

In [24]:
data2=data2[data2['기준연도']==2020]

## 1) 전처리

In [25]:
data2.isnull().sum()

기준연도         0
시도명          0
시도코드         0
시군구명         0
시군구코드        0
상권업종대분류명     0
상권업종중분류명     0
상권업종소분류명     0
상권업종소분류코드    0
상권업종개수       0
dtype: int64

In [26]:
data2

,기준연도,시도명,시도코드,시군구명,시군구코드,상권업종대분류명,상권업종중분류명,상권업종소분류명,상권업종소분류코드,상권업종개수
51075,2020,서울특별시,11,강남구,11680,관광/여가/오락,PC/오락/당구/볼링등,기원,N01A09,10
51076,2020,서울특별시,11,강남구,11680,관광/여가/오락,PC/오락/당구/볼링등,당구장,N01A03,124
51077,2020,서울특별시,11,강남구,11680,관광/여가/오락,PC/오락/당구/볼링등,볼링장,N01A06,1
51078,2020,서울특별시,11,강남구,11680,관광/여가/오락,PC/오락/당구/볼링등,비디오감상실,N01A02,12
51079,2020,서울특별시,11,강남구,11680,관광/여가/오락,PC/오락/당구/볼링등,인터넷PC방,N01A01,153
...,...,...,...,...,...,...,...,...,...,...
61286,2020,서울특별시,11,중랑구,11260,학문/교육,학원-컴퓨터,컴퓨터학원,R06A01,6
61287,2020,서울특별시,11,중랑구,11260,학문/교육,학원기타,학원(종합),R09A01,121
61288,2020,서울특별시,11,중랑구,11260,학문/교육,학원기타,학원-기타,R09A11,24
61289,2020,서울특별시,11,중랑구,11260,학문/교육,학원기타,학원-실내운전,R09A04,1


### 1> 불필요 항목 제거

In [27]:
data2.drop(columns=['시군구코드','상권업종소분류코드','시도코드'], inplace=True)

In [28]:
data2

,기준연도,시도명,시군구명,상권업종대분류명,상권업종중분류명,상권업종소분류명,상권업종개수
51075,2020,서울특별시,강남구,관광/여가/오락,PC/오락/당구/볼링등,기원,10
51076,2020,서울특별시,강남구,관광/여가/오락,PC/오락/당구/볼링등,당구장,124
51077,2020,서울특별시,강남구,관광/여가/오락,PC/오락/당구/볼링등,볼링장,1
51078,2020,서울특별시,강남구,관광/여가/오락,PC/오락/당구/볼링등,비디오감상실,12
51079,2020,서울특별시,강남구,관광/여가/오락,PC/오락/당구/볼링등,인터넷PC방,153
...,...,...,...,...,...,...,...
61286,2020,서울특별시,중랑구,학문/교육,학원-컴퓨터,컴퓨터학원,6
61287,2020,서울특별시,중랑구,학문/교육,학원기타,학원(종합),121
61288,2020,서울특별시,중랑구,학문/교육,학원기타,학원-기타,24
61289,2020,서울특별시,중랑구,학문/교육,학원기타,학원-실내운전,1


In [30]:
data2['상권업종대분류명'].unique()

array(['관광/여가/오락', '부동산', '생활서비스', '소매', '숙박', '스포츠', '음식', '학문/교육'],
      dtype=object)

In [32]:
data2[data2['상권업종대분류명']=='스포츠']

,기준연도,시도명,시군구명,상권업종대분류명,상권업종중분류명,상권업종소분류명,상권업종개수
51392,2020,서울특별시,강남구,스포츠,실내운동시설,기타실내운동시설,5
51393,2020,서울특별시,강남구,스포츠,실내운동시설,복싱도장,2
51394,2020,서울특별시,강남구,스포츠,실내운동시설,스쿼시장,1
51395,2020,서울특별시,강남구,스포츠,실내운동시설,스포츠센터,7
51396,2020,서울특별시,강남구,스포츠,실내운동시설,실내골프연습장,15
...,...,...,...,...,...,...,...
61159,2020,서울특별시,중랑구,스포츠,실내운동시설,스포츠센터,1
61160,2020,서울특별시,중랑구,스포츠,실내운동시설,실내골프연습장,2
61161,2020,서울특별시,중랑구,스포츠,실내운동시설,이종격투기,2
61162,2020,서울특별시,중랑구,스포츠,실내운동시설,체육관,3
